In [0]:
import numpy as np
import pandas as pd

In [7]:
#upload file
from google.colab import files
uploaded = files.upload()

Saving data_test_JAN24_2020.csv to data_test_JAN24_2020.csv


In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['data_test_JAN24_2020.csv']))
vals = df.values

Preprocess data set.

In [0]:
# get specific values for each cell 
data = []
for row in vals:
  strs = row[0].split('|')
  data.append(strs)
# generate a new dataframe 
cols = df.columns.values[0].split('|')[1:]
cols.insert(0,"ID")
newDF = pd.DataFrame(data, columns=cols)


In [0]:
newDF.head()

,ID,clicks,complaints,hard_bounces,opens,rejects,sent,soft_bounces,template,time,unique_clicks,unique_opens,unsubs
0,187,0,1,0,0,0,0,0,NEW COLLECTION - MAY 2018,5/23/2018 0:00,0,0,0
1,188,0,0,0,0,0,0,1,NEW COLLECTION - MAY 2018,5/21/2018 1:00,0,0,0
2,189,0,0,0,0,0,0,1,NEW COLLECTION - MAY 2018,5/21/2018 0:00,0,0,0
3,190,0,0,0,0,0,0,10,NEW COLLECTION - MAY 2018,5/20/2018 23:00,0,0,0
4,191,0,0,0,0,0,0,1,NEW COLLECTION - MAY 2018,5/20/2018 21:00,0,0,0


This is the function that returns result.

In [0]:
def getRes(templates,inputDF):
  # only pick records whose templates are chosen ones
  dfPicked = inputDF[(inputDF['template'].isin(templates))]
  # calculate overall results
  sentCount = 0
  rejectCount = 0
  openCount = 0
  clickCount = 0
  for index, row in dfPicked.iterrows():
    sentCount += int(row['sent'])
    rejectCount += int(row['rejects'])
    openCount += int(row['opens'])
    clickCount += int(row['clicks'])
  deliveredCount = sentCount-rejectCount
  print ("sent = %d" % (sentCount))
  print ("delivered = %d" % (deliveredCount))  
  print ("delivery rate = %f" % (deliveredCount/sentCount if sentCount>0 else 0))  
  print ("open rate = %f" % (openCount/deliveredCount if deliveredCount>0 else 0))  
  print ("click rate = %f" % (clickCount/deliveredCount if deliveredCount>0 else 0))  
  dfSorted = dfPicked.copy()
  dfSorted['time'] = pd.to_datetime(dfSorted['time']).dt.normalize()
  # only pick events that happened in 2018 and 2019
  dfSorted=dfSorted[(dfSorted['time']>=pd.datetime(2018,1,1))&(dfSorted['time']<=pd.datetime(2019,12,31))]
  dfSorted=dfSorted.set_index('time')
  dfSorted['sentCount'] = dfSorted['sent'].astype('int')
  dfSorted['rejectCount'] = dfSorted['rejects'].astype('int')
  dfSorted['openCount'] = dfSorted['opens'].astype('int')
  dfSorted['clickCount'] = dfSorted['clicks'].astype('int')
  # merge events in the same month
  dfMerge=dfSorted.resample('M').sum()
  dfMerge['deliveredCount'] = dfMerge['sentCount']-dfMerge['rejectCount']
  dfMerge['delivery rate'] = dfMerge['deliveredCount']/dfMerge['sentCount']
  dfMerge['open rate'] = dfMerge['openCount']/dfMerge['deliveredCount']
  dfMerge['clickrate'] = dfMerge['clickCount']/dfMerge['deliveredCount']
  dfMerge = dfMerge[(dfMerge['sentCount']>0)]
  dfMerge = dfMerge.to_period('M')
  return dfMerge

You can input the templates

In [15]:
inputTemp = input()

NEW COLLECTION - MAY 2018,WINTER SALE - JAN 2016


In [0]:
candidates=(inputTemp.split(','))

In [23]:
dfRes=getRes(candidates,newDF)
print(dfRes)

sent = 29599
delivered = 29167
delivery rate = 0.985405
open rate = 0.486132
click rate = 0.068708
         sentCount  rejectCount  openCount  ...  delivery rate  open rate  clickrate
time                                        ...                                     
2018-05      19387          181       8105  ...       0.990664   0.422004   0.059356

[1 rows x 8 columns]


In [22]:
possibleTemplates = sorted(list(set(newDF['template'].values)))
for each in possibleTemplates:
  print(each)

NEW COLLECTION - MAY 2018
NEW COLLECTION - OCTOBER 2018
NJ-WAREHOUSE-NOV2018
PICTO-C1-NOV-2018
PICTO-C2-NOV-2018
PICTO-C3-NOV-2018
PICTO-C4-NOV-2018
PICTO-C5-NOV-2018
PICTO-C6-NOV-2018
PICTO-LABORDAY-2019
PICTO-OPTIN-AUG-2019
PICTO-P1-NOV-2018
PICTO-P2-NOV-2018
PICTO-P3-NOV-2018
PICTO-P4-NOV-2018
PICTO-P5-NOV-2018
PICTO-P6-NOV-2018
PICTO-REACTIVATION-PROSPECTS-JAN25-2019
RB - PERFORMANCE - Collectors - S3
RB ONBOARDING E2 NE
RB Onboarding E1 CALI
RB Onboarding E1 CANADA ENG
RB Onboarding E1 CANADA FRA
RB Onboarding E1 DCFL
RB Onboarding E1 DCFL TEST
RB Onboarding E1 NE
RB Onboarding E1-June-2016
RB Onboarding E2 CALI
RB Onboarding E2 CALI TEST
RB Onboarding E2 CANADA ENG
RB Onboarding E2 CANADA FRA
RB Onboarding E2 DCFL
RB Onboarding E2 DCFL TEST
RB Onboarding E2-June-2016
RB Onboarding E3 NE
RB Onboarding E3-June-2016
RB Onboarding E4 CALI
RB Onboarding E4 CALI TEST
RB Onboarding E4 CANADA ENG
RB Onboarding E4 CANADA FRA
RB Onboarding E4 DCFL
RB Onboarding E4 NE
RB Onboarding E4-June-